# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from vacation_api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,asau,46.4333,26.4000,51.96,68,99,1.41,RO,1651467799
1,1,lebu,-37.6167,-73.6500,50.49,93,3,12.91,CL,1651467799
2,2,bredasdorp,-34.5322,20.0403,54.57,93,98,5.03,ZA,1651467800
3,3,mar del plata,-38.0023,-57.5575,51.87,71,100,10.00,AR,1651467800
4,4,hilo,19.7297,-155.0900,74.28,100,100,8.99,US,1651467613
...,...,...,...,...,...,...,...,...,...,...
527,527,jibuti,11.5000,42.5000,80.08,57,44,9.60,DJ,1651467743
528,528,plymouth,41.8334,-70.8328,50.72,79,0,0.00,US,1651468042
529,529,prabumulih,-3.4500,104.2500,90.61,56,95,4.18,ID,1651468043
530,530,almaty,43.2500,76.9500,76.91,38,40,4.47,KZ,1651468043


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
gmaps.configure(api_key=g_key)

In [44]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) 
    & (city_data_df["Wind Speed"] < 10) 
    & (city_data_df["Cloudiness"] == 0) ]

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
164,164,arraial do cabo,-22.9661,-42.0278,74.66,75,0,7.09,BR,1651467868
201,201,vila velha,-20.3297,-40.2925,76.95,82,0,8.05,BR,1651467478
209,209,cape canaveral,28.4058,-80.6048,76.98,80,0,9.22,US,1651467888
264,264,saint pete beach,27.7253,-82.7412,73.00,84,0,3.44,US,1651467909
290,290,buraydah,26.3260,43.9750,79.83,15,0,8.30,SA,1651467921
294,294,aswan,24.0934,32.9070,76.30,19,0,5.75,EG,1651467507
312,312,do gonbadan,30.3586,50.7981,76.28,35,0,2.73,IR,1651467932
326,326,tarpon springs,28.1461,-82.7568,72.95,90,0,0.00,US,1651467744
336,336,morondava,-20.2833,44.2833,72.05,70,0,8.93,MG,1651467947
369,369,canberra,-35.2835,149.1281,70.68,49,0,6.91,AU,1651467962


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"]=""
hotel_df

C:\Users\Garnin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
164,arraial do cabo,BR,-22.9661,-42.0278,
201,vila velha,BR,-20.3297,-40.2925,
209,cape canaveral,US,28.4058,-80.6048,
264,saint pete beach,US,27.7253,-82.7412,
290,buraydah,SA,26.3260,43.9750,
294,aswan,EG,24.0934,32.9070,
312,do gonbadan,IR,30.3586,50.7981,
326,tarpon springs,US,28.1461,-82.7568,
336,morondava,MG,-20.2833,44.2833,
369,canberra,AU,-35.2835,149.1281,


In [47]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params)
    response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("No data")

hotel_df

C:\Users\Garnin\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Hotel Name
164,arraial do cabo,BR,-22.9661,-42.0278,Pousada Porto Praia
201,vila velha,BR,-20.3297,-40.2925,Hotel Vitória Palace
209,cape canaveral,US,28.4058,-80.6048,Residence Inn by Marriott Cape Canaveral Cocoa...
264,saint pete beach,US,27.7253,-82.7412,Grand Plaza Hotel - St. Pete Beach
290,buraydah,SA,26.3260,43.9750,Mövenpick Qassim
294,aswan,EG,24.0934,32.9070,Nuba Nile
312,do gonbadan,IR,30.3586,50.7981,شبکه بهداشت و درمان گچساران
326,tarpon springs,US,28.1461,-82.7568,"Innisbrook, A Salamander Golf and Spa Resort"
336,morondava,MG,-20.2833,44.2833,La Case Bambou
369,canberra,AU,-35.2835,149.1281,Novotel Canberra


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…